## Get Playlists Data

Let us understand how to get YouTube Playlists Data using Python as programming language using REST APIs.

Here are the steps involved in getting the data.
* Make sure to get the token and build **youtube** object.
* We can get list of playlists using `youtube.playlists().list` with appropriate arguments.
* As per the table structure below we want playlist id, title, description, video count and privacy status.
  * Playlist id, title and description comes from snippet part.
  * Count comes from contentDetails part.
  * Privacy Status comes from status part.
* We need to pass all the 3 parts to part argument for `youtube.playlists().list` along with channel id.

```python
playlists = youtube.playlists().list(
    part='snippet,contentDetails,status',
    channelId='UCakdSIPsJqiOLqylgoYmwQg'
).execute()
```

* YouTube APIs return at max 50 results. We need to capture nextToken and invoke `list` function to get next 50 results.

```python
nextPageToken = None
playlist_details = []
while True:
    playlists = youtube.playlists().list(
        part = 'snippet,contentDetails,status',
        channelId='UCakdSIPsJqiOLqylgoYmwQg',
        pageToken = nextPageToken,
        maxResults = 50
    ).execute()

    for item in playlists['items']:
        playlist_details.append(
            (
                item['id'], item['snippet']['title'], 
                item['snippet']['description'],
                item['contentDetails']['itemCount'], 
                item['status']['privacyStatus']
            )
        )

    nextPageToken = playlists.get('nextPageToken')
    if not nextPageToken:
        break
```

### Playlists Table Structure

Here are the fields we have as part of playlists table.

|Column Name|Data Type|
|---|---|
|playlist_id|VARCHAR(50)|
|playlist_title|VARCHAR(200)|
|playlist_description|VARCHAR(5000)|
|playlist_video_count|INT|
|playlist_privacy_status|VARCHAR(30)|

In [ ]:
%run 03_configure_youtube_apis.ipynb    

In [ ]:
youtube

In [ ]:
playlists = youtube.playlists().list(
    part='snippet,contentDetails,status',
    channelId=channel_id
).execute()

In [ ]:
playlists['items'][0]

In [ ]:
nextPageToken = None
playlist_details = []
while True:
    playlists = youtube.playlists().list(
        part = 'snippet,contentDetails,status',
        channelId='UCakdSIPsJqiOLqylgoYmwQg',
        pageToken = nextPageToken,
        maxResults = 50
    ).execute()

    for item in playlists['items']:
        playlist_details.append(
            (
                item['id'], item['snippet']['title'], item['snippet']['description'],
                item['contentDetails']['itemCount'], item['status']['privacyStatus']
            )
        )

    nextPageToken = playlists.get('nextPageToken')
    if not nextPageToken:
        break

In [ ]:
len(playlist_details)

In [ ]:
playlist_details[0]

In [ ]:
def get_playlists(youtube, channel):
    nextPageToken = None
    playlists_details = []
    
    while True:
        playlists = youtube.playlists().list(
            part = 'snippet,contentDetails,status',
            channelId=channel,
            pageToken = nextPageToken,
            maxResults = 50
        ).execute()

        for item in playlists['items']:
            playlists_details.append(
                (
                    item['id'], item['snippet']['title'], item['snippet']['description'],
                    item['contentDetails']['itemCount'], item['status']['privacyStatus']
                )
            )

        nextPageToken = playlists.get('nextPageToken')
        if not nextPageToken:
            break

    return playlists_details

In [ ]:
def get_playlist_video_details(playlist_item):
    snippet = playlist_item['snippet']
    return (
        snippet.get('playlistId'), 
        snippet['resourceId'].get('videoId'), 
        snippet.get('position'), 
        snippet.get('publishedAt')
    )

In [ ]:
def get_playlist_items(playlistId, channelId):
    nextPageToken = None
    video_ids = []
    while True:
        playlistItems = youtube.playlistItems().list(
            part = 'snippet',
            playlistId = playlistId,
            pageToken = nextPageToken,
            maxResults = 50
        ).execute()
        playlist_items = []
        for item in playlistItems['items']:
            if item['snippet']['channelId'] == channelId:
                playlist_items.append(get_playlist_video_details(item))
        video_ids += playlist_items
        
        nextPageToken = playlistItems.get('nextPageToken')
        if not nextPageToken:
            break
    return video_ids